Preparing Data process. 
Outcome: 
- Features extracted 
- columns concatanated
- Labels marked
- Data parsing
- ready for vectorization
- New python objects or csv files created for easier access

# Table of Content
- Imports
- Load Data
- Methods/Global/Constants
- kaggle_fakenews_df preparing
- kaggle_huff_df preparing
- kaggle_new_york_times_df preparing
- buzzfeed_fakenews_df preparing
- Combined Data

# Import

In [24]:
from bs4 import BeautifulSoup
import datetime
import json
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle
import requests
import time
from html.parser import HTMLParser
import lxml
from lxml.html.clean import Cleaner
import re
from random import randint
from zipfile import ZipFile

# Load Cleaned Data

In [5]:
_  = os.path.join('data', 'clean_kaggle_fakenews_train_df.pkl')
with open(_, 'rb') as file:
    kaggle_fakenews_prep_df = pickle.load(file)

In [7]:
_  = os.path.join('data', 'clean_huff_df.pkl')
with open(_, 'rb') as file:
    kaggle_huff_prep_df = pickle.load(file)

In [14]:
_  = os.path.join('data', 'clean_nytimes_df.pkl')
with open(_, 'rb') as file:
    kaggle_new_york_times_prep_df = pickle.load(file)

In [11]:
_  = os.path.join('data', 'clean_buzzfeed_fakenews_df.pkl')
with open(_, 'rb') as file:
    buzzfeed_fakenews_prep_df = pickle.load(file)

# Methods/Global/Constants

In [ ]:
def clean_me(url):
    time.sleep(randint(0, 3))
        
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',
    }
    
    try:
        html  = requests.get(url, headers=headers).text
    except:
        return 'none'
    
    soup = BeautifulSoup(html, "html.parser") # create a new bs4 object from the html data loaded
    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    text = text.replace('\n', ' ')
    text = text.replace('|', ' ')
    return text

# kaggle_fakenews_df preparing

In [101]:
# Combining all text to one big string
kaggle_fakenews_prep_df['total'] = kaggle_fakenews_df.drop(columns=['label']).values.sum(axis=1)
#kaggle_fakenews_prep_df.to_pickle("data/prep_kaggle_fakenews_train_df.pkl")

# kaggle_huff_df preparing

In [17]:
kaggle_huff_prep_df.drop(columns=['date', 'category'], inplace=True)

#One time call
# Scrape articles from all the links & save/add to txt file
# file_path  = os.path.join('data', 'scraped_huff_articles.txt')
# outF = open(file_path, "a", encoding="utf-8") 
# for index, value in enumerate(kaggle_huff_prep_df['link']): 
#     outF.write(clean_me(value)) 
#     outF.write("\n")  
#     # Progress Confirmation
#     if index%10000 == 0:
#         print(datetime.datetime.now(), 'index', index)     
# outF.close()      

# Use the saved articles file to fill i  the dataframe column "articles"
file_path = os.path.join('data', 'scraped_huff_articles.txt')
with open(file_path, 'r', encoding="utf-8") as file:
    _ =  file.readlines()   
    
# Some simple parsing and manipulation
kaggle_huff_prep_df['text'] =  pd.Series(_)
kaggle_huff_prep_df.fillna(' ', inplace=True)
kaggle_huff_prep_df['total'] = kaggle_huff_prep_df.drop(columns=['link']).values.sum(axis=1)
kaggle_huff_prep_df['label'] = 0


# Due to time constraint only 2167 articles were scrapped.
#kaggle_huff_prep_df.iloc[:2167].to_pickle("data/prep_huff_df.pkl")

# kaggle_new_york_times_df preparing

In [20]:
kaggle_new_york_times_prep_df['label'] = 0

kaggle_new_york_times_prep_df['total'] = kaggle_new_york_times_prep_df['total'].map(lambda x: x.replace('\n', ''))

#kaggle_new_york_times_prep_df.to_pickle("data/prep_nytimes_df.pkl")

# buzzfeed_fakenews_df preparing

In [ ]:
# Scrape articles from all the urls for articles
# file_path  = os.path.join('data', 'buzzfeed_fake_articles.txt')
# outF = open(file_path, "a", encoding="utf-8") 
# for index, value in enumerate(buzzfeed_fakenews_prep_df['url']): 
#     outF.write(clean_me(value)) 
#     outF.write("\n")
#     if index%100 == 0:
#         print(datetime.datetime.now(), 'index', index)     
# outF.close()      


#Make a new text column using all the scrap articles
file_path = os.path.join('data', 'buzzfeed_fake_articles.txt')
with open(file_path, 'r', encoding="utf-8") as file:
    _ =  file.readlines()    
    
buzz_feed_fakenews_df['text'] =  pd.Series(_) 
    
# Make a new author column and set it equal to the domain of the url
buzzfeed_fakenews_prep_df['author'] = buzzfeed_fakenews_prep_df['url'].map(lambda x: x.split('/')[2])

#Fill any null values with empty string they're all fake so it doesn't matter
buzzfeed_fakenews_prep_df.fillna(' ', inplace=True)

#Remove column to prepare to merge with kaggle dataset
buzzfeed_fakenews_prep_df.drop(columns=['url', 'fb_engagement', 'published_date', 'category'], inplace=True)

#Create a total column a concatatation of everything and a label
buzzfeed_fakenews_prep_df['total'] = buzzfeed_fakenews_prep_df.values.sum(axis=1)
buzzfeed_fakenews_prep_df['label'] = 1

#Conform to the kaggle dataset column ordering
_ = kaggle_fakenews_prep_df.columns
buzzfeed_fakenews_prep_df = buzzfeed_fakenews_prep_df.reindex(columns=_)


#buzz_feed_fakenews_df.to_pickle("data/prep_buzzfeed_fakenews.pkl")

- The articles text isn't in the dataset but the urls are. 
- I perform automatic webscraping using the urls. I save each article as one long string and place it on one line in the txt file. Each line is one article. I read all lines into a list and make a new column called text with this list of long strings
- Every observation in this dataset is a fake news article so I decided to just fill any nulls with empty strings to presevre it
- A new column called labels is of value 1
- Author was just called the website host due to the difficulty of scraping to get the author name for each website

# Combined Data

In [22]:
#Extract prepared dataframes and combine them
_ = os.path.join('data', 'prepared_data_frames_1.zip') 
with ZipFile(_, 'r') as zip: 
    zip.extractall('data')

_ = os.path.join('data', 'prepared_data_frames_2.zip') 
with ZipFile(_, 'r') as zip: 
    zip.extractall('data')


# Get all prep dataframes in a list and combine them
tmp = [pickle.load(open(os.path.join('data', 'prep_kaggle_fakenews_train_df.pkl'), 'rb')),
       pickle.load(open(os.path.join('data', 'prep_huff_df.pkl'), 'rb')),
       pickle.load(open(os.path.join('data', 'prep_buzzfeed_fakenews_df.pkl'), 'rb')),
       pickle.load(open(os.path.join('data', 'prep_nytimes_df.pkl'), 'rb')),
      ]

final_df = pd.concat([tmp[0][['total', 'label']], tmp[1][['total', 'label']], tmp[2][['total', 'label']], tmp[3]])

#Saving the finalized dataframe
# final_df.to_pickle("data/prep_final_df.pkl")
# _ = os.path.join('data', 'prepared_data_frames_3.zip', 'data') 
# with ZipFile(_, 'w') as zip:
#     zip.write("data/prep_final_df.pkl")